In [114]:
#Installing packages and loading them into the enviroment
install.packages("MASS")
library("MASS")
install.packages("missMDA")
library("missMDA")
install.packages("tidyverse")
library("tidyverse")
install.packages("caret")
library("caret")
install.packages("mice")
library("mice")

Installing package into 'C:/Users/daeda/OneDrive/Documents/R/win-library/3.6'
(as 'lib' is unspecified)

Warning message:
"package 'MASS' is in use and will not be installed"
Installing package into 'C:/Users/daeda/OneDrive/Documents/R/win-library/3.6'
(as 'lib' is unspecified)

Warning message:
"package 'missMDA' is in use and will not be installed"
Installing package into 'C:/Users/daeda/OneDrive/Documents/R/win-library/3.6'
(as 'lib' is unspecified)

Warning message:
"package 'tidyverse' is in use and will not be installed"
Installing package into 'C:/Users/daeda/OneDrive/Documents/R/win-library/3.6'
(as 'lib' is unspecified)

Warning message:
"package 'caret' is in use and will not be installed"
Installing package into 'C:/Users/daeda/OneDrive/Documents/R/win-library/3.6'
(as 'lib' is unspecified)

Warning message:
"package 'mice' is in use and will not be installed"


In [229]:
#Loading all needed files,dropping first two coloumns, which are not needed for analysis (ID, species)
ionomics <- read.csv('spec_shoot_xyz_combined.csv', colClasses =c("NULL","NULL",NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,
                                                                    NA,NA,NA,NA,NA,NA,NA))
#Imputation via missMDA
imputeIonomics <- imputePCA(ionomics, method = "Regularized")

In [230]:
#Imputation via Mice
temp <- mice(ionomics, m = 5, maxit = 5, method = "norm.boot", seed = 123)
ionomics <- complete(temp,1)


 iter imp variable
  1   1  spec_as_int  S34  As75
  1   2  spec_as_int  S34  As75
  1   3  spec_as_int  S34  As75
  1   4  spec_as_int  S34  As75
  1   5  spec_as_int  S34  As75
  2   1  spec_as_int  S34  As75
  2   2  spec_as_int  S34  As75
  2   3  spec_as_int  S34  As75
  2   4  spec_as_int  S34  As75
  2   5  spec_as_int  S34  As75
  3   1  spec_as_int  S34  As75
  3   2  spec_as_int  S34  As75
  3   3  spec_as_int  S34  As75
  3   4  spec_as_int  S34  As75
  3   5  spec_as_int  S34  As75
  4   1  spec_as_int  S34  As75
  4   2  spec_as_int  S34  As75
  4   3  spec_as_int  S34  As75
  4   4  spec_as_int  S34  As75
  4   5  spec_as_int  S34  As75
  5   1  spec_as_int  S34  As75
  5   2  spec_as_int  S34  As75
  5   3  spec_as_int  S34  As75
  5   4  spec_as_int  S34  As75
  5   5  spec_as_int  S34  As75


In [231]:
# Split the data into training (80%) and test set (20%)
set.seed(123)
training.samples <- ionomics[,2] %>%
  createDataPartition(p = 0.8, list = FALSE)
train.data <- ionomics[training.samples, ]
test.data <- ionomics[-training.samples, ]

training.samples1 <- imputeIonomics$completeObs[,2] %>%
  createDataPartition(p = 0.8, list = FALSE)
train.data1 <- imputeIonomics$completeObs[training.samples1, ]
test.data1 <- imputeIonomics$completeObs[-training.samples1, ]
train.data1 <- unlist(train.data1)
test.data1 <- unlist(test.data1)
train.data1 <- as.data.frame(train.data1)
test.data1 <- as.data.frame(test.data1)

In [233]:
# Fit the model
model <- lda(spec_as_int~., data = train.data)
# Make predictions
predictions <- model %>% predict(test.data)
# Model accuracy
mean(predictions$class==test.data$spec_as_int)
#model

model1 <- lda(spec_as_int~., data = train.data1)
# Make predictions
predictions1 <- model1 %>% predict(test.data1)
# Model accuracy
mean(predictions1$class == test.data1$spec_as_int)
#model1

[1] 0.6290323

[1] 0.5080645

In [90]:
# Predicted classes
head(predictions$class, 6)
# Predicted probabilities of class memebership.
head(predictions$posterior, 6) 
# Linear discriminants
head(predictions$x, 3) 

[1] 1 6 6 6 3 3
Levels: 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24

,1,2,3,4,5,6,7,8,9,10,...,15,16,17,18,19,20,21,22,23,24
1,6.614597e-01,5.801399e-07,3.852542e-09,2.118628e-06,1.297464e-12,0.1801926728,1.583364e-01,2.899551e-08,5.425522e-06,1.531494e-10,...,4.453775e-07,1.501502e-09,3.180918e-17,6.537837e-18,1.039128e-25,1.362617e-24,2.603666e-20,4.671049e-15,7.015303e-14,5.286361e-16
2,3.905931e-02,1.064673e-01,3.707088e-04,1.593896e-01,8.105357e-08,0.3845877689,1.766937e-01,6.663725e-02,3.168168e-02,1.805296e-03,...,1.971091e-02,3.652453e-03,7.020411e-11,8.903824e-10,2.289822e-19,4.577419e-16,3.236151e-12,7.846109e-11,2.080221e-08,2.664764e-08
4,3.871845e-01,3.203659e-03,3.015328e-06,9.510502e-04,1.029806e-13,0.4546613897,1.480601e-01,2.220851e-04,1.901027e-03,2.221856e-05,...,2.288946e-03,1.303084e-05,4.367616e-14,3.291772e-12,8.679938e-20,1.518806e-18,1.350047e-16,2.095681e-12,2.041565e-10,1.455299e-10
5,1.625143e-03,1.881582e-01,2.041216e-04,2.235196e-01,8.627883e-09,0.3127684167,1.068556e-02,2.140431e-01,7.037698e-04,4.184055e-05,...,4.125475e-02,7.794320e-04,6.815265e-10,4.740077e-08,2.963817e-14,1.180236e-12,1.819868e-13,2.397883e-08,7.679657e-08,1.088796e-06
22,2.079059e-07,2.623519e-02,8.798524e-01,8.361407e-02,4.253858e-07,0.0001123531,3.131363e-05,1.015382e-02,3.145013e-08,4.867313e-10,...,2.389217e-08,1.878987e-08,2.331418e-10,2.444050e-12,8.567523e-18,2.769098e-16,1.521252e-11,4.300126e-12,1.268776e-10,8.345624e-11
25,6.979195e-08,1.676750e-01,4.399476e-01,3.627216e-01,3.952898e-09,0.0001475751,1.516285e-03,1.321077e-02,1.490595e-06,4.975061e-06,...,2.170871e-04,7.291984e-05,2.586530e-12,1.235864e-11,1.897457e-12,2.816463e-15,1.546992e-11,6.507968e-11,3.374646e-09,3.504048e-09


,LD1,LD2,LD3,LD4,LD5,LD6,LD7,LD8,LD9,LD10,LD11,LD12,LD13,LD14,LD15,LD16,LD17,LD18,LD19
1,-4.745348,1.3913653,1.0987070,2.0128129,0.83022331,2.9211019,-1.234070,0.4146289,1.6865190,0.95196188,-1.0382461,-1.4030587,1.370534,-0.8454875,-2.1036005,0.1588884,1.1114109,1.430900,-0.1118352
2,-3.341123,0.3905007,-0.9326754,0.4359576,-0.04149446,-0.3126965,-2.039649,-0.6376718,0.9997919,-0.02968626,0.2340459,-0.7529169,1.089833,-0.3655601,-0.2698029,-0.1502862,-0.1574731,2.093894,-1.3982138
4,-3.707217,0.4528865,0.1883600,0.1090248,1.28614956,1.9330317,-1.058791,-1.5646021,1.2759444,-0.22550831,0.2058524,-0.3646431,1.252327,-1.1456762,-1.8633902,0.2359874,-0.1598891,1.190433,0.1972949


|species|spec_as_int|
|---|---|
|acerifolia_x|1|
|aestivalis_x|2|
|cinerea_x|3|
|labrusca_x|4|
|palmata_x|5|
|riparia_x|6|
|rupestris_x|7|
|vulpina_x|8|
|acerifolia_y|9|
|aestivalis_y|10|
|cinerea_y|11|
|labrusca_y|12|
|palmata_y|13|
|riparia_y|14|
|rupestris_y|15|
|vulpina_y|16|
acerifolia_z|17|
|aestivalis_z|18|
|cinerea_z|19|
|labrusca_z|20|
|palmata_z|21|
|riparia_z|22|
|rupestris_z|23|
|vulpina_z|24|

In [123]:
table <- table(Predicted=predictions$class, Species=test.data$spec_as_int)
print(confusionMatrix(table))

ERROR: Error in !all.equal(nrow(data), ncol(data)): invalid argument type
